In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt 
import datetime as dt
import pandas as pd
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from scipy.optimize import minimize
import matplotlib.backends.backend_pdf
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import pickle
import matplotlib.colors as Colors
from numba import jit
from multiprocessing import Pool
import os
import Rheology_fitting_toolkit as rft
from sklearn.metrics import mean_squared_error
import math
import creep_part_identify as cpi

/tmp/ipykernel_242063/4207224591.py:16: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

dictionary_all_events = {}
CREEPMETER = ['XHR']
for q in range(len(CREEPMETER)):
    print(CREEPMETER[q])
    tm, min10_creep, tm2, min10_creep2 = rft.import_text(CREEPMETER[q])

    if CREEPMETER[q] == 'XSJ' or CREEPMETER[q] == 'XHR' or CREEPMETER[q] == 'XPK':
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)
        tm_int2, min10_creep_int2 = rft.interpolate(tm2,min10_creep2,CREEPMETER)
    elif CREEPMETER[q] == 'XMR':
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)
        tm_int2, min10_creep_int2 = rft.interpolate_1min(tm2,min10_creep2,CREEPMETER)
    else:
        tm_int, min10_creep_int = rft.interpolate(tm,min10_creep,CREEPMETER)


    df_PICKS, duration, START = rft.creepmeter_events(CREEPMETER[q])

    if CREEPMETER[q] == 'XSJ' or CREEPMETER[q] == 'XHR' or CREEPMETER[q] == 'XPK':
        data1  = rft.vel_acc(tm_int,min10_creep_int,10/60)
        data2 = rft.vel_acc(tm_int2,min10_creep_int2,10/60)
        data = data1.append(data2,ignore_index=True)
    elif CREEPMETER[q] == 'XMR':
        data1  = rft.vel_acc(tm_int,min10_creep_int,10/60)
        data2 = rft.vel_acc_1min(tm_int2,min10_creep_int2,1/60)
        data = data1.append(data2,ignore_index=True)
    else:
        data = rft.vel_acc(tm_int,min10_creep_int,10/60)


    df_auto = rft.parkfield_remover(df_PICKS,CREEPMETER[q])


    df_rain_day_total = rft.rain_timeseries(CREEPMETER[q])

    df_auto = rft.when_does_it_rain(df_auto,CREEPMETER[q])
    
    if CREEPMETER[q] == 'CWN':
        C_matrix = np.load('../../Rheology/CWN/CWN_covariance_matrix_12days_18_APR_23.npy')
        C_matrix_inv_CWN = np.linalg.inv(C_matrix)
    
    if CREEPMETER[q] == 'XHR':
        C_matrix_2 = np.load('../../Rheology/XHR/XHR_2_covariance_matrix_4days_27_APR_23.npy')
        C_matrix_3 = np.load('../../Rheology/XHR/XHR_3_covariance_matrix_4days_27_APR_23.npy')
        C_matrix_inv_2 = np.linalg.inv(C_matrix_2)
        C_matrix_inv_3 = np.linalg.inv(C_matrix_3)

    
    dataframes, creep_index = rft.creep_event_dataframe(df_auto,duration, START, data,CREEPMETER[q])

    Creep_phases = pd.read_csv("../../Rheology/{k}/Creep_phases_{k}.csv".format(k=CREEPMETER[q]),index_col=0)
    

XHR


In [3]:
import importlib
importlib.reload(rft)

<module 'Rheology_fitting_toolkit' from '/home/users/exet4136/CREEPMETER_DATA/SCRIPTS/Onset/Rheology_fitting_toolkit.py'>

In [ ]:
def LNV_fitter(time,slip,cov_matrix_inv,no_phases,columns_LNV,j,CREEPMETER,LNV_DF_params,atest,file_misfit):
    print('Linear Viscous: {k}'.format(k=j))
    print ("Current date and time : ")
    isExistLNV = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/LNV/{k}_{y}_LNV_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    if not isExistLNV:
        success = False
        n_iter = 5000
        LNV_bounds = LNV_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/LNV/{k}_{u}_LNV_fit_dictionary_multi_phase_SLSQP_28_APR_23.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            LNV_SLSQP = pickle.load(fname)
        LNV_initial_guess = LNV_SLSQP['fitting params'].loc['fitted'].to_list()
        while success == False:
            if success == False:
                res_LNV = scipy.optimize.basinhopping(rft.LNV_dromedary, LNV_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_LNV,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':LNV_bounds,'options':{'maxiter':n_iter}}),niter=1000)            
                n_iter = n_iter+2000
                success = res_LNV.success
                LNV_initial_guess = res_LNV.x
        dictionary_LNV = {}
        dictionary_LNV['fit'] = res_LNV
        LNV_fitting_params = pd.DataFrame([res_LNV.x],columns = ('Ts','Vs','K','T01','S1','Tau1','V01','T02','S2'), index = ['fitted'])
        LNV_DF_params = pd.concat([LNV_DF_params,LNV_fitting_params])
        dictionary_LNV['fitting params'] = LNV_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/LNV/{k}_{u}_LNV_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_LNV,tf)
    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/LNV/{k}_{u}_LNV_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_LNV = pickle.load(tf)
            res_LNV = dictionary_LNV['fit']
            LNV_DF_params = dictionary_LNV['fitting params']
    return LNV_DF_params

###################################################################################################
def PLV_fitter(time,slip,cov_matrix_inv,no_phases,columns_PLV,j,CREEPMETER,PLV_DF_params,atest,file_misfit):
    print('Power-law Viscous: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistPLV = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/PLV/{k}_{y}_PLV_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    if not isExistPLV:
        success = False
        n_iter = 5000
        PLV_bounds = PLV_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/PLV/{k}_{u}_PLV_fit_dictionary_multi_phase_SLSQP_28_APR_23.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            PLV_SLSQP = pickle.load(fname)
        PLV_initial_guess = PLV_SLSQP['fitting params'].loc['fitted'].to_list()
        while success == False:
            if success == False:
                res_PLV = scipy.optimize.basinhopping(rft.PLV_dromedary, PLV_initial_guess,\
                accept_test= atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_PLV,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':PLV_bounds,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_PLV.success
                PLV_initial_guess = res_PLV.x

        dictionary_PLV = {}
        dictionary_PLV['fit'] = res_PLV
        PLV_fitting_params = pd.DataFrame([res_PLV.x],columns = ('Ts','Vs','K','T01','S1','Tau1','V01','n1','T02','S2'), index = ['fitted'])
        PLV_DF_params = pd.concat([PLV_DF_params,PLV_fitting_params])
        dictionary_PLV['fitting params'] = PLV_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/PLV/{k}_{u}_PLV_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_PLV,tf)
    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/PLV/{k}_{u}_PLV_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_PLV = pickle.load(tf)
            res_PLV = dictionary_PLV['fit']
            PLV_DF_params = dictionary_PLV['fitting params']
    return PLV_DF_params
###################################################################################################
def VSF_SS_fitter(time,slip,cov_matrix_inv,no_phases,columns_VSF_SS,j,CREEPMETER,VSF_SS_DF_params,atest,file_misfit):
    print('VSF-SS: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistVSF_SS = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/VSF_SS/{k}_{y}_VSF_SS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    if not isExistVSF_SS:
        success = False
        n_iter = 5000
        VSF_SS_bounds = VSF_SS_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_SS/{k}_{u}_VSF_SS_fit_dictionary_multi_phase_SLSQP_28_APR_23.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            VSF_SS_SLSQP = pickle.load(fname)
        VSF_SS_initial_guess = VSF_SS_SLSQP['fitting params'].loc['fitted'].to_list()        
        while success == False:
            if success == False:
                res_VSF_SS = scipy.optimize.basinhopping(rft.VSF_SS_dromedary, VSF_SS_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_VSF_SS,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':VSF_SS_bounds}))#,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_VSF_SS.success
                VSF_SS_initial_guess = res_VSF_SS.x

        dictionary_VSF_SS = {}
        dictionary_VSF_SS['fit'] = res_VSF_SS
        VSF_SS_fitting_params = pd.DataFrame([res_VSF_SS.x],columns = ('Ts','Vs','K','T01','S1','Tau1','V01','T02','S2'), index = ['fitted'])
        VSF_SS_DF_params = pd.concat([VSF_SS_DF_params,VSF_SS_fitting_params])
        dictionary_VSF_SS['fitting params'] = VSF_SS_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_SS/{k}_{u}_VSF_SS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_VSF_SS,tf)
    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_SS/{k}_{u}_VSF_SS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_VSF_SS = pickle.load(tf)
            res_VSF_SS = dictionary_VSF_SS['fit']
            VSF_SS_DF_params = dictionary_VSF_SS['fitting params']
    return VSF_SS_DF_params
###################################################################################################
def RDF_fitter(time,slip,cov_matrix_inv,no_phases,columns_RDF,j,CREEPMETER,RDF_DF_params,atest,file_misfit):
    print('RDF: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistRDF = os.path.exists('../../Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/RDF/{k}_{y}_RDF_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    if not isExistRDF:
        success = False
        n_iter = 5000
        RDF_bounds = RDF_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/RDF/{k}_{u}_RDF_SS_fit_dictionary_multi_phase_SLSQP_28_APR_23.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            RDF_SLSQP = pickle.load(fname)
        RDF_initial_guess = RDF_SLSQP['fitting params'].loc['fitted'].to_list() 
        while success == False:
            if success == False:
                res_RDF = scipy.optimize.basinhopping(rft.RDF_dromedary, RDF_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_RDF,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':RDF_bounds,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_RDF.success
                RDF_initial_guess = res_RDF.x
        dictionary_RDF = {}
        dictionary_RDF['fit'] = res_RDF
        RDF_fitting_params = pd.DataFrame([res_RDF.x],columns = ('Ts','Vs','K','T01','S1','Ta1','V01','V0/VL1','T02','S2'), index = ['fitted'])
        RDF_DF_params = pd.concat([RDF_DF_params,RDF_fitting_params])
        dictionary_RDF['fitting params'] = RDF_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/RDF/{k}_{u}_RDF_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_RDF,tf)

    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/RDF/{k}_{u}_RDF_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_RDF = pickle.load(tf)
            res_RDF = dictionary_RDF['fit']
            RDF_DF_params = dictionary_RDF['fitting params']
    return RDF_DF_params
###################################################################################################
def VSF_aSS_fitter(time,slip,cov_matrix_inv,no_phases,columns_VSF_aSS,j,CREEPMETER,VSF_aSS_DF_params,atest,file_misfit):
    print('VSF_aSS: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistVSF_aSS = os.path.exists('../../Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/VSF_aSS/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    if not isExistVSF_aSS:
        #print('../../Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/VSF_aSS/{k}_{y}_VSF_aSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER[q],y=j))
        success = False
        n_iter = 5000
        VSF_aSS_bounds = VSF_aSS_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_SLSQP_28_APR_23.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            VSF_aSS_SLSQP = pickle.load(fname)        
        VSF_aSS_initial_guess = VSF_aSS_SLSQP['fitting params'].loc['fitted'].to_list() 
        while success == False:
            if success == False:
                res_VSF_aSS = scipy.optimize.basinhopping(rft.VSF_aSS_dromedary, VSF_aSS_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_VSF_aSS,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':VSF_aSS_bounds}))#,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_VSF_aSS.success
                VSF_aSS_initial_guess = res_VSF_aSS.x
        dictionary_VSF_aSS = {}
        dictionary_VSF_aSS['fit'] = res_VSF_aSS
        VSF_aSS_fitting_params = pd.DataFrame([res_VSF_aSS.x],columns = ('Ts','Vs','K','T01','S1','Ta1','V01','t1','T02','S2'), index = ['fitted'])
        VSF_aSS_DF_params = pd.concat([VSF_aSS_DF_params,VSF_aSS_fitting_params])
        dictionary_VSF_aSS['fitting params'] = VSF_aSS_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_VSF_aSS,tf)

    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_VSF_aSS = pickle.load(tf)
            res_VSF_aSS = dictionary_VSF_aSS['fit']
            VSF_aSS_DF_params = dictionary_VSF_aSS['fitting params']
    return VSF_aSS_DF_params
###################################################################################################
def VSF_bSS_fitter(time,slip,cov_matrix_inv,no_phases,columns_VSF_bSS,j,CREEPMETER,VSF_bSS_DF_params,atest,file_misfit):
    print('VSF-bSS: {k}'.format(k=j))
    print ("Current date and time : ")
    print (dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    isExistVSF_bSS = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/VSF_bSS/{k}_{y}_VSF_bSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER,y=j))
    if not isExistVSF_bSS:
        success = False
        n_iter = 5000
        VSF_bSS_bounds = VSF_bSS_DF_params.loc['bounds'].to_list()
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/SLSQP/{u}/VSF_bSS/{k}_{u}_VSF_bSS_fit_dictionary_multi_phase_SLSQP_28_APR_23.txt".format(k=CREEPMETER,u=j),'rb') as fname:
            VSF_bSS_SLSQP = pickle.load(fname)
        VSF_bSS_initial_guess = VSF_bSS_SLSQP['fitting params'].loc['fitted'].to_list() 
        while success == False:
            if success == False:
                res_VSF_bSS = scipy.optimize.basinhopping(rft.VSF_bSS_dromedary, VSF_bSS_initial_guess,\
                accept_test = atest, minimizer_kwargs = ({'args':(time,slip,cov_matrix_inv,no_phases,\
                columns_VSF_bSS,j,CREEPMETER,file_misfit),'method':'Nelder-Mead','bounds':VSF_bSS_bounds}))#,'options':{'maxiter':n_iter}}),niter=1000)
                n_iter = n_iter+2000
                success = res_VSF_bSS.success
                VSF_bSS_initial_guess = res_VSF_bSS.x

        dictionary_VSF_bSS = {}
        dictionary_VSF_bSS['fit'] = res_VSF_bSS
        VSF_bSS_fitting_params = pd.DataFrame([res_VSF_bSS.x],columns = ('Ts','Vs','K','T01','S1','Tau1','V01','A_B1','T02','S2'), index = ['fitted'])
        VSF_bSS_DF_params = pd.concat([VSF_bSS_DF_params,VSF_bSS_fitting_params])
        dictionary_VSF_bSS['fitting params'] = VSF_bSS_DF_params
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_bSS/{k}_{u}_VSF_bSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "wb") as tf:
            pickle.dump(dictionary_VSF_bSS,tf)
    else:
        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_bSS/{k}_{u}_VSF_bSS_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER,u=j), "rb") as tf:
            dictionary_VSF_bSS = pickle.load(tf)
            res_VSF_SS = dictionary_VSF_bSS['fit']
            VSF_bSS_DF_params = dictionary_VSF_bSS['fitting params']
    return VSF_bSS_DF_params


In [ ]:
def test_multi(j):
    print(j)
    if len(dataframes[j].Time) <= 577:
        isExist = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/figures/Nelder-Mead/{y}/{k}_{y}_event_plot_Multi_phase_Nelder-Mead_01_MAY_23.pdf'.format(k=CREEPMETER[q],y=j))
        if not isExist:
            isExist2 = os.path.exists('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/{k}_{y}_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER[q],y=j))
            if not isExist2:
                print('/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{y}/{k}_{y}_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt'.format(k=CREEPMETER[q],y=j))
                Creep_Phase_no = Creep_phases.iloc[j].dropna()
                number_of_phases = (len(Creep_Phase_no)/2)-1
                if 1<= number_of_phases <=2:
                    data_P0, data_P1, data_P2, data_P3, data_P4, creep_phase_new = rft.phase_splitter(Creep_Phase_no,dataframes[j])

                    LNV_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'LNV')
                    PLV_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'PLV')
                    VSF_SS_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'VSF_SS')
                    VSF_bSS_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'VSF_bSS')
                    VSF_aSS_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'VSF_aSS')
                    RDF_DF_params = rft.initial_and_bounds(creep_phase_new,data_P0,data_P1,data_P2,data_P3,data_P4,'RDF')

                    if CREEPMETER[q] == 'XHR':
                        if j < 70:
                            C_matrix_inv = C_matrix_inv_2
                        else:
                            C_matrix_inv = C_matrix_inv_3
                    else:
                        C_matrix_inv = C_matrix_inv_CWN

                    rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}".format(k=CREEPMETER[q]))
                    Rheologies_to_test = ['LNV','PLV','VSF_SS','VSF_bSS','VSF_aSS','RDF']#,'cb77']
                    if number_of_phases==1:
                        LNV_DF_params.drop(['Tau2','V02','T03','S3','Tau3','V03','T04','S4','Tau4','V04'], axis=1, inplace=True)
                        PLV_DF_params.drop(['Tau2','V02','n2','T03','S3','Tau3','V03','n3','T04','S4','Tau4','V04','n4'], axis=1, inplace=True)
                        VSF_SS_DF_params.drop(['Tau2','V02','T03','S3','Tau3','V03','T04','S4','Tau4','V04'], axis=1, inplace=True)
                        VSF_bSS_DF_params.drop(['Tau2','V02','A_B2','T03','S3','Tau3','V03','A_B3','T04','S4','Tau4','V04','A_B4'], axis=1, inplace=True)
                        VSF_aSS_DF_params.drop(['Ta2','V02','t2','T03','S3','Ta3','V03','t3','T04','S4','Ta4','V04','t4'], axis=1, inplace=True)
                        RDF_DF_params.drop(['Ta2','V02','V0/VL2','T03','S3','Ta3','V03','V0/VL3','T04','S4','Ta4','V04','V0/VL4'], axis=1, inplace=True)


                        for z in range(len(Rheologies_to_test)):
                            rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/{x}".format(k=CREEPMETER[q],u=j,x=Rheologies_to_test[z]))

                        LNV_params_tried = np.zeros(len(LNV_DF_params.loc['initial'].index.tolist()))
                        PLV_params_tried = np.zeros(len(PLV_DF_params.loc['initial'].index.tolist()))
                        VSF_SS_params_tried = np.zeros(len(VSF_SS_DF_params.loc['initial'].index.tolist()))
                        VSF_bSS_params_tried = np.zeros(len(VSF_bSS_DF_params.loc['initial'].index.tolist()))
                        VSF_aSS_params_tried = np.zeros(len(VSF_aSS_DF_params.loc['initial'].index.tolist()))
                        RDF_params_tried = np.zeros(len(RDF_DF_params.loc['initial'].index.tolist()))

                        columns_LNV = LNV_DF_params.loc['initial'].index.tolist()
                        columns_PLV = PLV_DF_params.loc['initial'].index.tolist()
                        columns_VSF_SS = VSF_SS_DF_params.loc['initial'].index.tolist()
                        columns_VSF_bSS = VSF_bSS_DF_params.loc['initial'].index.tolist()
                        columns_VSF_aSS = VSF_aSS_DF_params.loc['initial'].index.tolist()
                        columns_RDF = RDF_DF_params.loc['initial'].index.tolist()

                        rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}".format(k=CREEPMETER[q],u=j))

                        def atest(f_new, x_new, f_old, x_old):
                            if f_old < f_new:
                                return False
                            else:
                                return True

                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/LNV/{k}_{u}_LNV_misfit_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER[q],u=j),'a') as file_LNV:
                            LNV_DF_params = LNV_fitter(np.array(dataframes[j].Time),np.array(dataframes[j].Slip), C_matrix_inv,number_of_phases,columns_LNV,j,CREEPMETER[q],LNV_DF_params,atest,file_LNV)

                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/PLV/{k}_{u}_PLV_misfit_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER[q],u=j),'a') as file_PLV:
                            PLV_DF_params = PLV_fitter(np.array(dataframes[j].Time),np.array(dataframes[j].Slip), C_matrix_inv,number_of_phases,columns_PLV,j,CREEPMETER[q],PLV_DF_params,atest,file_PLV)

                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_SS/{k}_{u}_VSF_SS_misfit_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER[q],u=j),'a') as file_VSF_SS:
                            VSF_SS_DF_params = VSF_SS_fitter(np.array(dataframes[j].Time),np.array(dataframes[j].Slip), C_matrix_inv,number_of_phases,columns_VSF_SS,j,CREEPMETER[q],\
                                                             VSF_SS_DF_params,atest,file_VSF_SS)

                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_bSS/{k}_{u}_VSF_bSS_misfit_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER[q],u=j),'a') as file_VSF_bSS:
                            VSF_bSS_DF_params = VSF_bSS_fitter(np.array(dataframes[j].Time),np.array(dataframes[j].Slip), C_matrix_inv,number_of_phases,columns_VSF_bSS,j,CREEPMETER[q],\
                                                               VSF_bSS_DF_params,atest,file_VSF_bSS)

                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/VSF_aSS/{k}_{u}_VSF_aSS_misfit_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER[q],u=j),'a') as file_VSF_aSS:
                            VSF_aSS_DF_params = VSF_aSS_fitter(np.array(dataframes[j].Time),np.array(dataframes[j].Slip), C_matrix_inv,number_of_phases,columns_VSF_aSS,j,CREEPMETER[q],\
                                                               VSF_aSS_DF_params,atest,file_VSF_aSS)

                        with open("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/fits/Nelder-Mead/{u}/RDF/{k}_{u}_RDF_misfit_Nelder-Mead_01_MAY_23.txt".format(k=CREEPMETER[q],u=j),'a') as file_RDF:
                            RDF_DF_params = RDF_fitter(np.array(dataframes[j].Time),np.array(dataframes[j].Slip), C_matrix_inv,number_of_phases,columns_RDF,j,CREEPMETER[q],RDF_DF_params,atest,file_RDF)
                        
                        test_t = np.arange(0,max(dataframes[j].Time.iloc[1:]),0.01)
                        rft.check_dir("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/figures/Nelder-Mead".format(k=CREEPMETER[q]))
                        
                        slip_LNV = rft.LNV_dromedary_plot(LNV_DF_params.loc['fitted'],test_t,number_of_phases)
                        slip_PLV = rft.PLV_dromedary_plot(PLV_DF_params.loc['fitted'],test_t,number_of_phases)
                        slip_VSF = rft.VSF_dromedary_plot(VSF_SS_DF_params.loc['fitted'],test_t,number_of_phases)
                        slip_VSF_bSS = rft.VSF_bSS_dromedary_plot(VSF_bSS_DF_params.loc['fitted'],test_t,number_of_phases)
                        slip_VSF_aSS = rft.VSF_aSS_dromedary_plot(VSF_aSS_DF_params.loc['fitted'],test_t,number_of_phases)
                        slip_RDF = rft.RDF_dromedary_plot(RDF_DF_params.loc['fitted'],test_t,number_of_phases)

                        colors = ['#74add1','#313695','#a50026','#d73027','#f46d43','#fdae61']

                        print('saving figure {k}'.format(k=j))        
                        plt.figure()
                        plt.scatter(dataframes[j].Time-2,dataframes[j].Slip, label='Observed Data',s=5,color='#000000', zorder=100)   
                        plt.plot(test_t-2,slip_LNV,label = 'Linear Viscous Flow', color = colors[0], linewidth=2, linestyle = '-')
                        plt.plot(test_t-2,slip_PLV,label='Power-law Viscous Flow', color = colors[1], linewidth=2, linestyle = '-')
                        plt.plot(test_t-2,slip_VSF,label = 'Velocity Strengthening Friction\n - steady state', color = colors[2], linewidth=2, linestyle = '-')
                        plt.plot(test_t-2,slip_VSF_aSS,label = 'R+S - Stress >> Steady State', color = colors[3], linewidth=2, linestyle = '-')
                        plt.plot(test_t-2,slip_VSF_bSS,label = 'R+S - Stress << Steady State', color = colors[4], linewidth=2, linestyle = '-')
                        plt.plot(test_t-2,slip_RDF,label = 'RDF', color = colors[5], linewidth=2, linestyle = '-')
                        plt.legend(fontsize=11)
                        plt.xlabel('Time since start of event, hrs', fontsize=14)
                        plt.ylabel('Slip, mm',fontsize=14)
                        plt.xticks(fontsize=14)
                        plt.yticks(fontsize=14)
                        plt.title('{K}_{P}'.format(K = CREEPMETER[q], P = j ),fontsize=18)
                        figure = plt.gcf()  # get current figure
                        figure.set_size_inches(6,6)
                        plt.tight_layout()
                        plt.savefig("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/figures/Nelder-Mead/{k}_{u}_event_plot_Multi_phase_Nelder-Mead_01_MAY_23.pdf".format(k=CREEPMETER[q],u=j),format='pdf')
                        plt.savefig("/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/{k}/figures/Nelder-Mead/{k}_{u}_event_plot_Multi_phase_Nelder-Mead_01_MAY_23.png".format(k=CREEPMETER[q],u=j),format='pdf')
                        plt.close('all')
    return

In [24]:
dictionary_all_events = {}
if __name__ == '__main__':
    with Pool(1) as pool:                         # Create a multiprocessing Pool           
        pool.map(test_multi, dataframes)

0
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/Nelder-Mead/0/XHR_0_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt
1
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/Nelder-Mead/1/XHR_1_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt
2
/home/earthquakes2/homes/Dan/Rheology/Single_rheology_28_APR_23/XHR/fits/Nelder-Mead/2/XHR_2_fit_dictionary_multi_phase_Nelder-Mead_01_MAY_23.txt
Linear Viscous: 2
Current date and time : 
2023-05-01 12:30:03


/home/earthquakes1/software/Ubuntu/anaconda3/envs/dan2/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'optimized_par' of function 'Linear_viscous'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "Rheology_fitting_toolkit.py", line 27:
@jit(nopython=True,error_model = 'numpy')
def Linear_viscous(optimized_par,OBS_Time):
^

  warnings.warn(NumbaPendingDeprecationWarning(msg, loc=loc))
/home/earthquakes1/software/Ubuntu/anaconda3/envs/dan2/lib/python3.10/site-packages/numba/core/ir_utils.py:2152: NumbaPendingDeprecationWarning: 
Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'optimized_par' of function 'Linear_viscous'.

For more information visit https://numba.readthedocs.io/en/stab

KeyboardInterrupt: 

In [43]:
def creep_event_dataframe_short(dataframe,df_auto):
    dataframes={}
    creep_index = np.array(df_auto.og_index)
    for j in range(len(dataframe)):
        boolarr = dataframe[j].Slip <= 0.8*max(dataframe[j].Slip)
        SLIP = dataframe[j].Slip[boolarr]
        TIME = dataframe[j].Time[boolarr]
        VEL = dataframe[j].Velocity[boolarr]
        ACC = dataframe[j].Acceleration[boolarr]
        dataframes[creep_index[j]] = pd.DataFrame({'Time':TIME,'Slip':SLIP,'Velocity':VEL,'Acceleration':ACC})
        dataframes[creep_index[j]].reset_index(inplace=True)
    return dataframes, creep_index